In [1]:
using Pkg
using LinearAlgebra
using Plots
using Distributions
using Random
using StateSpaceDynamics
using StatsBase

const SSD = StateSpaceDynamics

StateSpaceDynamics

In [2]:
"""
Create an underlying GaussianHMM to generate data
"""

# Create Guassian Emission Models
output_dim = 2
μ = [0.0, 0.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_1 = GaussianEmission(Gaussian(output_dim=output_dim, μ=μ, Σ=Σ))

μ = [2.0, 1.0]
Σ = 0.1 * Matrix{Float64}(I, output_dim, output_dim)
emission_2 = GaussianEmission(Gaussian(μ=μ, Σ=Σ, output_dim=output_dim))

# Create GaussianHMM
true_model = SSD.GaussianHMM(K=2, output_dim=2)
true_model.B[1] = emission_1
true_model.B[2] = emission_2
true_model.A = [0.9 0.1; 0.8 0.2]

# Sample from the model
n=1000
true_labels, data = SSD.sample(true_model, n=n)

([1, 1, 1, 1, 1, 1, 1, 2, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [-0.24987588432479124 -0.04689425260460763; 0.007459913013807148 -0.16555198059425713; … ; 0.12041547747978548 -0.041644650267985944; -0.5015985870405232 0.12172859381133226])

In [3]:
"""
Create a new GaussianHMM and try to fit the synthetic data
"""
n = 10  # Number of samples per trial
num_trials = 3  # Number of trials
trial_inputs = Vector{Matrix{Float64}}(undef, num_trials)  # Vector to store data matrices
trial_labels = Vector{Vector{Int}}(undef, num_trials)  # Vector to store label vectors
trial_outputs = Vector{Matrix{Float64}}(undef, num_trials)

for i in 1:num_trials
    true_labels, data = SSD.sample(true_model, n=n)  # Generate data and labels
    trial_labels[i] = true_labels  # Store labels for the ith trial
    trial_inputs[i] = data  # Store data matrix for the ith trial

    true_labels, data = SSD.sample(true_model, n=n)  # Generate data and labels
    trial_outputs[i] = data  # Store data matrix for the ith trial
end

In [6]:
"""
Generate a multiple trial GaussianHMM Dataset from the fit model
"""
model2 =SSD.GaussianHMM(K=2, output_dim=2)
ll = SSD.fit!(model2, trial_inputs, trials=true, max_iters=100)

74-element Vector{Float64}:
 -Inf
 -80.02593616709049
 -53.71576399874769
 -53.50268023616104
 -53.157603159417455
 -52.69831364030862
 -52.284161756845606
 -51.99661524077764
 -51.80185127049626
 -51.66162220200639
   ⋮
 -48.580042895490145
 -48.379716384478066
 -48.330421238078216
 -48.32261519313336
 -48.32168500502476
 -48.32158177837702
 -48.321569460001385
 -48.32156773359358
 -48.32156743805467

In [7]:
model3 = SSD.SwitchingGaussianRegression(K=2, input_dim=2, output_dim=2)
ll2 = SSD.fit!(model3, trial_inputs, trial_outputs, trials=true, max_iters=100)

10-element Vector{Float64}:
 -Inf
 -98.00143926441129
 -74.19960357423763
 -74.54636202011973
 -69.15361231393082
 -63.24368692678478
 -46.223622363438004
  73.31887967204617
 102.8519763643304
 102.8519763643304